#### Importando as bibliotecas

In [1]:
import pandas as pd
import xlrd
import pickle #modulo usado para serializar dados para o arquivo binario
import os
import time

#### Criando as funções necessárias para acessar as árvores TRIE

In [3]:
#Definindo classe de nodos TRIE
class TrieNode(object):
    def __init__(self, char: str):   # __init__ é um método especial para fazer construtores
        self.char = char   # caractere do nodo atual
        self.filhos = []   # nodos filhos
        self.pFinalizada = False   # se é o último nodo e a palavra terminou
        self.indices = []   # lista vazia para nodos que não são término de palavra
        ### não é um índice único e sim uma lista, pois podem existir títulos de música repetidos com mais de um artista
        
#Definindo função que adiciona uma string nova a arvore
def addString(raiz, palavra: str, indice: int):   # função usada para adicionar uma palavra nova à estrutura trie

    nodo = raiz
    for char in palavra:
        encontradoEmFilho = False
        # busca pelo caractere nos filhos do nodo atual
        for filho in nodo.filhos:
            if filho.char == char: 
                nodo = filho   # apontamos o nodo para o filho que contém esse char
                encontradoEmFilho = True
                break
        
        if not encontradoEmFilho:   # se o caractere não foi encontrado, adiciona novo filho
            novoNodo = TrieNode(char)
            nodo.filhos.append(novoNodo)
            nodo = novoNodo   # apontamos, então, o nodo para seu novo filho e continuamos a iteração
    
    nodo.pFinalizada = True   # indica que até ali pode ser uma palavra (nome de música/artista completo)
    nodo.indices.append(indice)   # assinala o indice passado
    
#Defiindo função que localiza uma string na arvore
def findString(raiz, palavra: str) -> (bool, []):
    """
      1. Se a palavra existe e tem algum índice associado, retorna verdadeiro e a lista de índices dela
      2. Se a palavra não existe, retorna falso e uma lista vazia
    """
    nodo = raiz

    if not raiz.filhos:   # se o nodo raiz não tiver nenhum filho, trivial, retorna falso
        return False, []
    for char in palavra:
        charNaoEncontrado = True
        for filho in nodo.filhos:
            if filho.char == char:
                charNaoEncontrado = False   # assinala que o char foi encontrado
                nodo = filho   # passa iteração para o nodo filho
                break
        
        if charNaoEncontrado:   # retorna falso quando não encontrar algum char da palavra sendo procurada
            return False, []
    
    # Caso passe por todos os caracteres sem retornar [False, []], então significa que a palavra foi encontrada
    # Resta saber se aquele nodo é um nodo final com um índice associado
    if nodo.pFinalizada:
        return True, nodo.indices
    else:
        return False, []

#### Fazendo a leitura dos arquivos

In [4]:
#Faz a leitura do arquivo de registros
with (open('teste.bin', 'rb')) as openfile:
    dados = pickle.load(openfile)
    
#Leitura do arquivo de indices de anos
with (open('indices.bin', 'rb')) as openfile:
    indices = pickle.load(openfile)

#Leitura do arquivo com a arvore TRIE de titulos de musicas
with open('trieMusicas.bin','rb') as openfile:
    trieMusicas = pickle.load(openfile)
        
#Leitura do arquivo com a arvore TRIE de nomes de artistas
with open('trieArtistas.bin','rb') as openfile:
    trieArtistas = pickle.load(openfile)

## Definição das Querys